# Data Wrangling

Merge and tidy repos and users tables to prepare for visualization.

In [1]:
import re

import pandas as pd

## Prepare Repo Data

Load the repos data and drop duplicates:

In [2]:
repos = pd.read_csv("data/2016/repos-dump.csv", quotechar='"', skipinitialspace=True)
print('Shape before dropping duplicates', repos.shape)
repos = repos.drop_duplicates(subset='full_name', keep='last')
print('Shape after  dropping duplicates', repos.shape)
repos.head()

Shape before dropping duplicates (8033, 5)
Shape after  dropping duplicates (8031, 5)


,full_name,stars,forks,description,language
0,yarnpkg/yarn,21050,784,"📦🐈 Fast, reliable, and secure dependency manag...",JavaScript
1,facebookincubator/create-react-app,17532,1819,Create React apps with no build configuration.,JavaScript
2,zeit/hyper,13605,907,A terminal built on web technologies,JavaScript
3,ParsePlatform/parse-server,12164,3319,Parse-compatible API server module for Node/Ex...,JavaScript
4,juliangarnier/anime,10239,537,Javascript Animation Engine,JavaScript


Separate out the `user` and `repo` from `full_name` into new columns:

In [3]:
def extract_user(line):
    return line.split('/')[0]

def extract_repo(line):
    return line.split('/')[1]

repos['user'] = repos['full_name'].str[:].apply(extract_user)
repos['repo'] = repos['full_name'].str[:].apply(extract_repo)
print(repos.shape)
repos.head()

(8031, 7)


,full_name,stars,forks,description,language,user,repo
0,yarnpkg/yarn,21050,784,"📦🐈 Fast, reliable, and secure dependency manag...",JavaScript,yarnpkg,yarn
1,facebookincubator/create-react-app,17532,1819,Create React apps with no build configuration.,JavaScript,facebookincubator,create-react-app
2,zeit/hyper,13605,907,A terminal built on web technologies,JavaScript,zeit,hyper
3,ParsePlatform/parse-server,12164,3319,Parse-compatible API server module for Node/Ex...,JavaScript,ParsePlatform,parse-server
4,juliangarnier/anime,10239,537,Javascript Animation Engine,JavaScript,juliangarnier,anime


## Prepare User Data

Load the users data and drop duplicates:

In [4]:
users = pd.read_csv("data/2016/user-geocodes-dump.csv", quotechar='"', skipinitialspace=True)
print('Shape before dropping duplicates', users.shape)
users = users.drop_duplicates(subset='id', keep='last')
print('Shape after  dropping duplicates', users.shape)
users.head()

Shape before dropping duplicates (5986, 8)
Shape after  dropping duplicates (5986, 8)


,id,name,type,location,lat,long,city,country
0,golastmile,LASTMILE,Organization,Berlin,52.520007,13.404954,Berlin,Germany
1,kmagiera,Krzysztof Magiera,User,NaN,NaN,NaN,NaN,NaN
2,RPISEC,NaN,Organization,"Troy, NY",42.728412,-73.691785,Troy,United States
3,camilatigre,Camila Tigre,User,Brazil,-14.235004,-51.925280,NaN,Brazil
4,felipevolpone,Felipe Volpone,User,Brazil,-14.235004,-51.925280,NaN,Brazil


Rename column `id` to `user`:

In [5]:
users.rename(columns={'id': 'user'}, inplace=True)
users.head()

,user,name,type,location,lat,long,city,country
0,golastmile,LASTMILE,Organization,Berlin,52.520007,13.404954,Berlin,Germany
1,kmagiera,Krzysztof Magiera,User,NaN,NaN,NaN,NaN,NaN
2,RPISEC,NaN,Organization,"Troy, NY",42.728412,-73.691785,Troy,United States
3,camilatigre,Camila Tigre,User,Brazil,-14.235004,-51.925280,NaN,Brazil
4,felipevolpone,Felipe Volpone,User,Brazil,-14.235004,-51.925280,NaN,Brazil


## Merge Repo and User Data

Left join repos and users:

In [6]:
repos_users = pd.merge(repos, users, on='user', how='left')
print('Shape repos:', repos.shape)
print('Shape users:', users.shape)
print('Shape repos_users:', repos_users.shape)
repos_users.head()

Shape repos: (8031, 7)
Shape users: (5986, 8)
Shape repos_users: (8031, 14)


,full_name,stars,forks,description,language,user,repo,name,type,location,lat,long,city,country
0,yarnpkg/yarn,21050,784,"📦🐈 Fast, reliable, and secure dependency manag...",JavaScript,yarnpkg,yarn,Yarn,Organization,NaN,NaN,NaN,NaN,NaN
1,facebookincubator/create-react-app,17532,1819,Create React apps with no build configuration.,JavaScript,facebookincubator,create-react-app,Facebook Incubator,Organization,"Menlo Park, California",37.452960,-122.181725,Menlo Park,United States
2,zeit/hyper,13605,907,A terminal built on web technologies,JavaScript,zeit,hyper,ZEIT,Organization,NaN,NaN,NaN,NaN,NaN
3,ParsePlatform/parse-server,12164,3319,Parse-compatible API server module for Node/Ex...,JavaScript,ParsePlatform,parse-server,Parse,Organization,"Menlo Park, CA",37.452960,-122.181725,Menlo Park,United States
4,juliangarnier/anime,10239,537,Javascript Animation Engine,JavaScript,juliangarnier,anime,Julian Garnier,User,Paris,48.856614,2.352222,Paris,France


## Tidy Up Repo and User Data

Re-order the columns:

In [7]:
repos_users = repos_users.reindex_axis(['full_name',
                                        'repo',
                                        'description',
                                        'stars',
                                        'forks',
                                        'language',
                                        'user',
                                        'name',
                                        'type',
                                        'location',
                                        'lat',
                                        'long',
                                        'city',
                                        'country'], axis=1)
print(repos_users.shape)
repos_users.head()

(8031, 14)


,full_name,repo,description,stars,forks,language,user,name,type,location,lat,long,city,country
0,yarnpkg/yarn,yarn,"📦🐈 Fast, reliable, and secure dependency manag...",21050,784,JavaScript,yarnpkg,Yarn,Organization,NaN,NaN,NaN,NaN,NaN
1,facebookincubator/create-react-app,create-react-app,Create React apps with no build configuration.,17532,1819,JavaScript,facebookincubator,Facebook Incubator,Organization,"Menlo Park, California",37.452960,-122.181725,Menlo Park,United States
2,zeit/hyper,hyper,A terminal built on web technologies,13605,907,JavaScript,zeit,ZEIT,Organization,NaN,NaN,NaN,NaN,NaN
3,ParsePlatform/parse-server,parse-server,Parse-compatible API server module for Node/Ex...,12164,3319,JavaScript,ParsePlatform,Parse,Organization,"Menlo Park, CA",37.452960,-122.181725,Menlo Park,United States
4,juliangarnier/anime,anime,Javascript Animation Engine,10239,537,JavaScript,juliangarnier,Julian Garnier,User,Paris,48.856614,2.352222,Paris,France


## Add Overall Ranks

Rank each element based on number of stars:

In [8]:
repos_users['rank'] = repos_users['stars'].rank(ascending=False)
print(repos_users.shape)
repos_users.head()

(8031, 15)


,full_name,repo,description,stars,forks,language,user,name,type,location,lat,long,city,country,rank
0,yarnpkg/yarn,yarn,"📦🐈 Fast, reliable, and secure dependency manag...",21050,784,JavaScript,yarnpkg,Yarn,Organization,NaN,NaN,NaN,NaN,NaN,2
1,facebookincubator/create-react-app,create-react-app,Create React apps with no build configuration.,17532,1819,JavaScript,facebookincubator,Facebook Incubator,Organization,"Menlo Park, California",37.452960,-122.181725,Menlo Park,United States,4
2,zeit/hyper,hyper,A terminal built on web technologies,13605,907,JavaScript,zeit,ZEIT,Organization,NaN,NaN,NaN,NaN,NaN,8
3,ParsePlatform/parse-server,parse-server,Parse-compatible API server module for Node/Ex...,12164,3319,JavaScript,ParsePlatform,Parse,Organization,"Menlo Park, CA",37.452960,-122.181725,Menlo Park,United States,9
4,juliangarnier/anime,anime,Javascript Animation Engine,10239,537,JavaScript,juliangarnier,Julian Garnier,User,Paris,48.856614,2.352222,Paris,France,16


## Verify Results: Users

Equivalent [GitHub search query](https://github.com/search?utf8=%E2%9C%93&q=created%3A2016-01-01..2016-12-31+stars%3A%3E%3D100+user%3Adonnemartin&type=Repositories&ref=searchresults): `created:2016-01-01..2016-12-31 stars:>=100 user:donnemartin`

*Note: The data might be slightly off, as the search query will take into account data up to when the query was executed.  Data in this notebook was mined on January 1, 2017 to 'freeze' the results for the year 2016.  The longer you run the search from January 1, 2017, the larger the discrepancy.*

In [9]:
repos_users[repos_users['user'] == 'donnemartin']

,full_name,repo,description,stars,forks,language,user,name,type,location,lat,long,city,country,rank
3684,donnemartin/gitsome,gitsome,A supercharged Git/GitHub command line interfa...,4482,158,Python,donnemartin,Donne Martin,User,"Washington, D.C.",38.907192,-77.036871,Washington,United States,89.0
3882,donnemartin/viz,viz,Interactive visualizations and stats of GitHub...,359,27,Python,donnemartin,Donne Martin,User,"Washington, D.C.",38.907192,-77.036871,Washington,United States,2515.5


## Verify Results: Python Repos

Equivalent [GitHub search query](https://github.com/search?utf8=%E2%9C%93&q=created%3A2016-01-01..2016-12-31+stars%3A%3E%3D100+language%3Apython&type=Repositories&ref=searchresults): `created:2016-01-01..2016-12-31 stars:>=100 language:python`

*Note: The data might be slightly off, as the search query will take into account data up to when the query was executed.  Data in this notebook was mined on January 1, 2017 to 'freeze' the results for the year 2016.  The longer you run the search from January 1, 2017, the larger the discrepancy.*

In [10]:
print(repos_users[repos_users['language'] == 'Python'].shape)
repos_users[repos_users['language'] == 'Python'].head()

(864, 15)


,full_name,repo,description,stars,forks,language,user,name,type,location,lat,long,city,country,rank
3674,tensorflow/models,models,Models built with TensorFlow,10327,2702,Python,tensorflow,NaN,Organization,NaN,NaN,NaN,NaN,NaN,15
3675,songrotek/Deep-Learning-Papers-Reading-Roadmap,Deep-Learning-Papers-Reading-Roadmap,Deep Learning papers reading roadmap for anyon...,8697,1122,Python,songrotek,Flood Sung,User,China,35.861660,104.195397,NaN,China,23
3676,Rochester-NRT/RocAlphaGo,RocAlphaGo,"An independent, student-led replication of Dee...",7597,2111,Python,Rochester-NRT,NaN,Organization,NaN,NaN,NaN,NaN,NaN,35
3677,alexjc/neural-doodle,neural-doodle,Turn your two-bit doodles into fine artworks w...,7225,514,Python,alexjc,Alex J. Champandard,User,"Vienna, Austria",48.208174,16.373819,Vienna,Austria,38
3678,p-e-w/maybe,maybe,:open_file_folder: :rabbit2: :tophat: See wha...,6078,162,Python,p-e-w,Philipp Emanuel Weidmann,User,Anywhere the Internet is,NaN,NaN,NaN,NaN,52


## Verify Results: Overall Repos

Equivalent [GitHub search query](https://github.com/search?utf8=%E2%9C%93&q=created%3A2016-01-01..2016-12-31+stars%3A%3E%3D100&type=Repositories&ref=searchresults): `created:2016-01-01..2016-12-31 stars:>=100`

*Note: The data might be slightly off, as the search query will take into account data up to when the query was executed.  Data in this notebook was mined on January 1, 2017 to 'freeze' the results for the year 2016.  The longer you run the search from January 1, 2017, the larger the discrepancy.*

In [11]:
print(repos_users.shape)
repos_users.head()

(8031, 15)


,full_name,repo,description,stars,forks,language,user,name,type,location,lat,long,city,country,rank
0,yarnpkg/yarn,yarn,"📦🐈 Fast, reliable, and secure dependency manag...",21050,784,JavaScript,yarnpkg,Yarn,Organization,NaN,NaN,NaN,NaN,NaN,2
1,facebookincubator/create-react-app,create-react-app,Create React apps with no build configuration.,17532,1819,JavaScript,facebookincubator,Facebook Incubator,Organization,"Menlo Park, California",37.452960,-122.181725,Menlo Park,United States,4
2,zeit/hyper,hyper,A terminal built on web technologies,13605,907,JavaScript,zeit,ZEIT,Organization,NaN,NaN,NaN,NaN,NaN,8
3,ParsePlatform/parse-server,parse-server,Parse-compatible API server module for Node/Ex...,12164,3319,JavaScript,ParsePlatform,Parse,Organization,"Menlo Park, CA",37.452960,-122.181725,Menlo Park,United States,9
4,juliangarnier/anime,anime,Javascript Animation Engine,10239,537,JavaScript,juliangarnier,Julian Garnier,User,Paris,48.856614,2.352222,Paris,France,16


## Output Results

Write out the results to csv to visualize in Tableau:

In [12]:
users.to_csv('data/2016/users.csv', index=False)
repos_users.to_csv('data/2016/repos-users-geocodes.csv', index=False)
repos_users.to_csv('data/2016/repos-users.csv', index=False)

In [13]:
repos_rank = repos_users.reindex_axis(['full_name', 'rank'], axis=1)
repos_rank.to_csv('data/2016/repos-ranks.csv', index=False)